# Crop YOLOv4 people

## Path definitions - change to your own

In [5]:
IMAGES_PATH = "/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedImgs/"
YOLOV4_RESULTS_CSV = "/content/drive/MyDrive/TFM/yolov4_results/transform_90.csv"
OUT_IMAGES = "/content/drive/MyDrive/TFM/yolov4_crop_90"

Here Google Drive storage is loaded. You can omit this cell if you are not using it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
%mkdir {OUT_IMAGES}

In [7]:
import pandas as pd
import cv2
import os

In [8]:
df = pd.read_csv(YOLOV4_RESULTS_CSV)
df

,filename,x,y,x2,y2,class
0,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,375,0,775,1079,person
1,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1374,189,1689,882,person
2,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,865,178,1091,865,person
3,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1573,173,1792,883,person
4,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,385,0,774,1079,person
...,...,...,...,...,...,...
7597,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,431,365,500,557,person
7598,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1316,346,1641,1077,person
7599,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1001,364,1149,788,person
7600,/content/drive/MyDrive/TFM/TGCRBNWv0.1UnifiedI...,1303,337,1595,1039,person


In [17]:
def crop_image(index, object_detection, save_path):
  img = cv2.imread(object_detection['filename'])
  crop_img = img[object_detection['y']:object_detection['y2'], object_detection['x']:object_detection['x2']]
  name = f'{index}_{os.path.basename(object_detection["filename"])}'
  cv2.imwrite(os.path.join(save_path, name), crop_img)

In [ ]:
for index, row in df.iterrows():
  dir_name = os.path.join(OUT_IMAGES, os.path.basename(row["filename"]).split('.jpg')[0])

  try:
    os.mkdir(dir_name)
    print(f'[+] Directory {dir_name} created') 
  except FileExistsError:
    pass

  crop_image(index, row, dir_name)

  print(f'[{index}] {os.path.join(dir_name, os.path.basename(row["filename"]))}')